# Cargo Loading Binary Classification 

## 1. Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Load Library

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras import backend as K

## 3. Load Data

In [ ]:
# // train_data path
train_path = "/content/drive/MyDrive/Colab_Notebooks/data/train/"
#// test_data path
test_path="/content/drive/MyDrive/Colab_Notebooks/data/test"

In [ ]:
labels=[]
label_paths = glob.glob(train_path+"*")
for t_path in label_paths:
    labels.append(t_path[len(train_path):])
labels.sort()
print(labels)

['0', '1']


In [ ]:
picture_size = 150
train_data = []
train_data_label =[]

for label in labels:
    print (label)
    img_files = glob.glob(train_path+label+"/*.*")
    num_label = labels.index(label)
    print("data size for " + label + " : " + str(len(img_files)))
    for img_file in img_files:
        #print("loading "+img_file)
        img = cv2.imread(img_file)
        img = cv2.resize(img,(picture_size,picture_size))
        train_data.append(img)
        train_data_label.append(num_label)

0
data size for 0 : 7748
1
data size for 1 : 4628


In [ ]:
x = np.array(train_data)
y = np.array(train_data_label)

X_train, X_val, y_train, y_val = train_test_split(x,y,test_size = 0.2, random_state=230)

Y_train = to_categorical(y_train, len(labels))
Y_val = to_categorical(y_val, len(labels))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [ ]:
val_test_datagen = ImageDataGenerator(rescale=1./255.)

In [ ]:
train_generator = train_datagen.flow(X_train,Y_train,batch_size=64)
validation_generator = val_test_datagen.flow(X_val,Y_val,batch_size=64)

## 3. Model

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

58889256/58889256 [==============================] - 2s 0us/step


In [ ]:
conv_base.trainable = True

In [ ]:
model = models.Sequential()
model.add(conv_base)

#model 에 Flatten층 추가
model.add(layers.Flatten())



model.add(layers.Dense(512, activation="relu"))
model.add(keras.layers.BatchNormalization())
model.add(layers.Dense(256, activation="relu"))
model.add(layers.Dropout(rate=0.1)) 
 
model.add(layers.Dense(128, activation="relu"))
model.add(keras.layers.BatchNormalization())

#model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(2, activation="softmax"))
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)


In [ ]:
model.compile(optimizer=optimizer,
              loss="binary_crossentropy",
              metrics=["acc"])

In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "/content/drive/My Drive/Colab_Notebooks/hjdt.h5",
    monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

In [ ]:

epoch_num =30
#모델 훈련 시 콜백함수 등록하여 호출
history = model.fit(train_generator,
      epochs=epoch_num,
      validation_data= validation_generator,
      callbacks=[checkpoint_cb]) #code here

Epoch 1/30
155/155 [==============================] - ETA: 0s - loss: 0.6674 - acc: 0.6745
Epoch 1: val_acc improved from -inf to 0.36874, saving model to /content/drive/My Drive/Colab_Notebooks/hjdt.h5
155/155 [==============================] - 104s 557ms/step - loss: 0.6674 - acc: 0.6745 - val_loss: 1.9780 - val_acc: 0.3687
Epoch 2/30
155/155 [==============================] - ETA: 0s - loss: 0.5424 - acc: 0.7481
Epoch 2: val_acc improved from 0.36874 to 0.37116, saving model to /content/drive/My Drive/Colab_Notebooks/hjdt.h5
155/155 [==============================] - 79s 508ms/step - loss: 0.5424 - acc: 0.7481 - val_loss: 1.4942 - val_acc: 0.3712
Epoch 3/30
155/155 [==============================] - ETA: 0s - loss: 0.4686 - acc: 0.7922
Epoch 3: val_acc improved from 0.37116 to 0.80937, saving model to /content/drive/My Drive/Colab_Notebooks/hjdt.h5
155/155 [==============================] - 80s 513ms/step - loss: 0.4686 - acc: 0.7922 - val_loss: 0.5243 - val_acc: 0.8094
Epoch 4/30
1

In [ ]:
picture_size = 150
test_data = []

img_files = glob.glob(test_path+"/*.*")

for img_file in img_files:
    img = cv2.imread(img_file)
    img = cv2.resize(img,(picture_size,picture_size))
    test_data.append(img)

val_test_datagen = ImageDataGenerator(rescale=1./255.)

## 4. Prediction

In [ ]:
import pandas as pd

df=pd.DataFrame({'Id':[], 'Class':[] })

model = tf.keras.models.load_model("/content/drive/My Drive/Colab_Notebooks/hjdt.h5")
for i in range(len(test_data)):
    test = np.array(test_data[i])
    test=test/255.0
    test=np.expand_dims(test, axis=0)  
    y_pred = model.predict(test)
    y_pred_num = np.argmax(y_pred, axis=1)[0]
    df.loc[i]=[i+1,y_pred_num]

df=df.set_index('Id')

1/1 [==============================] - 0s 18ms/step


In [ ]:
df

,Class
Id,
1.0,1.0
2.0,0.0
3.0,1.0
4.0,1.0
5.0,1.0
...,...
1824.0,1.0
1825.0,1.0
1826.0,0.0


## 5. Submission

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab_Notebooks/cargo_pred.csv',sep=',', encoding='cp949')